In [22]:
import warnings

from nacl.pwhash.argon2i import verify
from sympy.solvers.diophantine.diophantine import Linear

warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
import numpy as np
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import train_test_split
import time
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import ComplementNB
from sklearn.preprocessing import MinMaxScaler
from io import StringIO
rng = np.random.RandomState(42)
import pandas as pd
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import TreeSearch, BayesianEstimator
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import ast
import ast

In [3]:
df = pd.read_csv("csv/outlier_filtered.csv")

response_var = 'Diabetes_012'
features = list(df.columns)
features.remove(response_var)

print(features, response_var)
# Pretty-print using tabulate
df.head(1)

['BMI_q_normal', 'MentHlth', 'PhysHlth_q_uniform', 'GenHlth_q_uniform', 'Age_q_uniform', 'Education_coxbox', 'Income_q_uniform', 'HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex'] Diabetes_012


,BMI_q_normal,MentHlth,PhysHlth_q_uniform,GenHlth_q_uniform,Age_q_uniform,Education_coxbox,Income_q_uniform,Diabetes_012,HighBP,HighChol,...,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,DiffWalk,Sex
0,1.60221,1.998592,0.891892,1.0,0.581582,-1.109347,0.117117,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [123]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

models = {
    'Logistic regression': BayesSearchCV(
        LogisticRegression(solver='newton-cholesky'),
        {
            'penalty': Categorical(['l2', None]),
            'C': Real(0.001, 100, prior='log-uniform'),
            'class_weight': Categorical(['balanced', None])
        },
        random_state=rng,
        scoring='f1_macro',
        n_jobs=-1,
        n_points=5,
        n_iter=50
    ),
    
    # Discriminant Analysis
    'Discriminant analysis (not svd)': BayesSearchCV(
        LinearDiscriminantAnalysis(),
        {
            'solver': Categorical(['lsqr', 'eigen']),
            'shrinkage': Real(0.0001, 1.0, prior='uniform'),
            'tol': Real(1e-5, 1.0, prior='uniform'),
        },
        random_state=rng,
        scoring='f1_macro',
        n_jobs=-1,
        n_points=4,
        n_iter=50
    ),
    # Discriminant Analysis
    'Discriminant analysis (svd)': BayesSearchCV(
        LinearDiscriminantAnalysis(store_covariance=True),
        {
            'tol': Real(1e-5, 1.0, prior='uniform'),
        },
        random_state=rng,
        scoring='f1_macro',
        n_jobs=-1,
        n_points=4,
        n_iter=50
    ),
    'QuadraticDiscriminantAnalysis': BayesSearchCV(
        QuadraticDiscriminantAnalysis(),
        {
            'reg_param': Real(0.0001, 1.0, prior='uniform'),  # Regularization parameter
            'tol': Real(1e-5, 1.0, prior='uniform'),          # Tolerance for convergence
        },
        random_state=rng,
        scoring='f1_macro',
        n_jobs=-1,
        n_points=4,
        n_iter=50
    ),
  'GradientBoostingClassifier': BayesSearchCV(
         GradientBoostingClassifier(random_state=rng),
        {
            'n_estimators': Integer(50, 200),  # Number of boosting stages (trees)
            'learning_rate': Real(0.01, 0.3, prior='uniform'),  # Learning rate
            'max_depth': Integer(3, 10),  # Maximum depth of individual estimators
            'min_samples_split': Integer(2, 10),  # Minimum number of samples required to split an internal node
            'subsample': Real(0.5, 1.0, prior='uniform'),  # Fraction of samples used for fitting each base learner
        },
        random_state=42,
        scoring='f1_macro',
        n_jobs=-1,
        n_iter=4,
        n_points=3
    ),
}

In [124]:
X, y = df[features], df[response_var]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.9,
                                                    random_state=rng,
                                                    stratify=y)

fea_subsets = {
    8:[0, 3, 4, 6, 7, 8, 12, 16],
    10:[0, 3, 4, 6, 7, 8, 9, 12, 16, 20],
    12:[0, 3, 4, 6, 7, 8, 9, 11, 12, 16, 17, 19],
    14:[0, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 16, 17, 19],
    16:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 16, 17, 1]
}


In [125]:
# Start tracking total runtime
def run_models(elapse=0, skip=[]):
    start_total = time.time()
    
    y = y_train
    it = 0
    
    for model in models.keys():
        if model not in ['GradientBoostingClassifier']:
            continue 
            
        for k in fea_subsets.keys():
            if it <= elapse or model in skip:
                print(f"Skipping it={it} with {model};{k};")
                it += 1
                continue 
                
            # Start tracking time for this iteration
            start_time = time.time()
            
            # Define the feature subset and input data
            fea_subset = np.array(features)[fea_subsets[k]]
            X = X_train[fea_subset]
            
            # Fit the model
            bayes_search = models[model]
            
            
            print(f"\rFitting {model}...                   ", end="")
            if model == 'ComplementNB':
                X_scaled = MinMaxScaler().fit_transform(X)
                bayes_search.fit(X_scaled, y)  # Fit the model on the transformed data
            else:
                bayes_search.fit(X, y) 
            print(f"\rFitting {model} completed            ", end="")
            
            
            # Get best parameters and score
            best_params = bayes_search.best_params_
            score = bayes_search.score(X_test[fea_subset], y_test)
            
            # Calculate elapsed time for this iteration
            elapsed_time = time.time() - start_time
            
            # Print results along with time taken
            print(f"\r{it};{model};{k};{best_params};{score:01.04f};{elapsed_time:06.02f}")
            it += 1
    
    # Calculate and print total runtime
    total_time = time.time() - start_total
    print(f"Total runtime: {total_time:.2f} seconds")

run_models(-1)

0;GradientBoostingClassifier;8;OrderedDict({'learning_rate': 0.25284262311045247, 'max_depth': 9, 'min_samples_split': 4, 'n_estimators': 193, 'subsample': 0.9320639577324754});0.3970;1788.74
1;GradientBoostingClassifier;10;OrderedDict({'learning_rate': 0.25284262311045247, 'max_depth': 9, 'min_samples_split': 4, 'n_estimators': 193, 'subsample': 0.9320639577324754});0.3972;1333.72
2;GradientBoostingClassifier;12;OrderedDict({'learning_rate': 0.25284262311045247, 'max_depth': 9, 'min_samples_split': 4, 'n_estimators': 193, 'subsample': 0.9320639577324754});0.4022;1343.06
3;GradientBoostingClassifier;14;OrderedDict({'learning_rate': 0.25284262311045247, 'max_depth': 9, 'min_samples_split': 4, 'n_estimators': 193, 'subsample': 0.9320639577324754});0.4033;1474.30
4;GradientBoostingClassifier;16;OrderedDict({'learning_rate': 0.25284262311045247, 'max_depth': 9, 'min_samples_split': 4, 'n_estimators': 193, 'subsample': 0.9320639577324754});0.4023;1566.56
Total runtime: 7506.38 seconds


In [126]:
output = """
it;model;num fea;best parameters;score;elapsed time 
0;Logistic regression;8;OrderedDict({'C': 0.009275062636581224, 'class_weight': 'balanced', 'penalty': 'l2'});0.4428;073.66
1;Logistic regression;10;OrderedDict({'C': 0.019884322615837377, 'class_weight': 'balanced', 'penalty': 'l2'});0.4421;073.19
2;Logistic regression;12;OrderedDict({'C': 0.0014661424746571484, 'class_weight': 'balanced', 'penalty': 'l2'});0.4420;081.25
3;Logistic regression;14;OrderedDict({'C': 0.0049931659015543745, 'class_weight': 'balanced', 'penalty': 'l2'});0.4424;097.71
4;Logistic regression;16;OrderedDict({'C': 0.008986806532830008, 'class_weight': 'balanced', 'penalty': 'l2'});0.4434;133.19
5;Discriminant analysis (not svd);8;OrderedDict({'shrinkage': 0.7753069475939367, 'solver': 'eigen', 'tol': 1.0});0.4343;038.70
6;Discriminant analysis (not svd);10;OrderedDict({'shrinkage': 0.7516415380327516, 'solver': 'lsqr', 'tol': 1e-05});0.4365;037.12
7;Discriminant analysis (not svd);12;OrderedDict({'shrinkage': 0.7709731236105747, 'solver': 'lsqr', 'tol': 1e-05});0.4437;026.60
8;Discriminant analysis (not svd);14;OrderedDict({'shrinkage': 0.7385508959093038, 'solver': 'lsqr', 'tol': 1e-05});0.4399;033.46
9;Discriminant analysis (not svd);16;OrderedDict({'shrinkage': 0.7820175034407315, 'solver': 'lsqr', 'tol': 1e-05});0.4283;048.99
10;Discriminant analysis (svd);8;OrderedDict({'tol': 0.8787898063269084});0.3933;029.79
11;Discriminant analysis (svd);10;OrderedDict({'tol': 0.8007791561388778});0.3934;044.66
12;Discriminant analysis (svd);12;OrderedDict({'tol': 0.881514287541885});0.4077;036.27
13;Discriminant analysis (svd);14;OrderedDict({'tol': 0.8682414474458714});0.4080;051.64
14;Discriminant analysis (svd);16;OrderedDict({'tol': 0.8804868604416194});0.3986;050.19
15;ComplementNB;8;OrderedDict({'alpha': 9.83097052187787, 'fit_prior': True, 'force_alpha': False, 'norm': False});0.3544;019.80
16;ComplementNB;10;OrderedDict({'alpha': 10.0, 'fit_prior': False, 'force_alpha': False, 'norm': False});0.3820;020.02
17;ComplementNB;12;OrderedDict({'alpha': 9.728726870982992, 'fit_prior': True, 'force_alpha': True, 'norm': True});0.3965;020.74
18;ComplementNB;14;OrderedDict({'alpha': 10.0, 'fit_prior': True, 'force_alpha': True, 'norm': True});0.4114;021.65
19;ComplementNB;16;OrderedDict({'alpha': 9.955213218146183, 'fit_prior': True, 'force_alpha': True, 'norm': True});0.4058;020.61
0;QuadraticDiscriminantAnalysis;8;OrderedDict({'reg_param': 0.0001, 'tol': 1e-05});0.4208;032.06
1;QuadraticDiscriminantAnalysis;10;OrderedDict({'reg_param': 0.003149940162422831, 'tol': 0.0029025043785303713});0.4409;032.99
2;QuadraticDiscriminantAnalysis;12;OrderedDict({'reg_param': 0.0001, 'tol': 0.8165426531203231});0.4290;039.92
3;QuadraticDiscriminantAnalysis;14;OrderedDict({'reg_param': 0.00029860625576866914, 'tol': 0.0848853341071096});0.4284;042.26
4;QuadraticDiscriminantAnalysis;16;OrderedDict({'reg_param': 0.0001, 'tol': 1.0});0.4330;050.90
0;GradientBoostingClassifier;8;OrderedDict({'learning_rate': 0.25284262311045247, 'max_depth': 9, 'min_samples_split': 4, 'n_estimators': 193, 'subsample': 0.9320639577324754});0.3970;1788.74
1;GradientBoostingClassifier;10;OrderedDict({'learning_rate': 0.25284262311045247, 'max_depth': 9, 'min_samples_split': 4, 'n_estimators': 193, 'subsample': 0.9320639577324754});0.3972;1333.72
2;GradientBoostingClassifier;12;OrderedDict({'learning_rate': 0.25284262311045247, 'max_depth': 9, 'min_samples_split': 4, 'n_estimators': 193, 'subsample': 0.9320639577324754});0.4022;1343.06
3;GradientBoostingClassifier;14;OrderedDict({'learning_rate': 0.25284262311045247, 'max_depth': 9, 'min_samples_split': 4, 'n_estimators': 193, 'subsample': 0.9320639577324754});0.4033;1474.30
4;GradientBoostingClassifier;16;OrderedDict({'learning_rate': 0.25284262311045247, 'max_depth': 9, 'min_samples_split': 4, 'n_estimators': 193, 'subsample': 0.9320639577324754});0.4023;1566.56
"""

stats = pd.read_csv(StringIO(output), delimiter=";")
print(stats.columns)
best_models = stats.loc[stats.groupby('model')['score'].idxmax()]
best_models

Index(['it', 'model', 'num fea', 'best parameters', 'score', 'elapsed time '], dtype='object')


,it,model,num fea,best parameters,score,elapsed time
18,18,ComplementNB,14,"OrderedDict({'alpha': 10.0, 'fit_prior': True,...",0.4114,21.65
7,7,Discriminant analysis (not svd),12,"OrderedDict({'shrinkage': 0.7709731236105747, ...",0.4437,26.60
13,13,Discriminant analysis (svd),14,OrderedDict({'tol': 0.8682414474458714}),0.4080,51.64
28,3,GradientBoostingClassifier,14,OrderedDict({'learning_rate': 0.25284262311045...,0.4033,1474.30
4,4,Logistic regression,16,"OrderedDict({'C': 0.008986806532830008, 'class...",0.4434,133.19
21,1,QuadraticDiscriminantAnalysis,10,OrderedDict({'reg_param': 0.003149940162422831...,0.4409,32.99


In [37]:
# import bnlearn as bn
# from sklearn.model_selection import GridSearchCV
# import pandas as pd
# from sklearn.metrics import f1_score
# 
# # Define the target or class variable
# class_node = response_var
# 
# # Initialize a dictionary to store model performance
# root_nodes = [col for col in df.columns if col != class_node]
# model_scores = {}
# 
# 
# # Validation set for scoring
# df_test = pd.concat([X_test, y_test.rename(class_node)], axis=1)



for k in fea_subsets.keys():
    if k < 16:
        continue 
    
    if k != 16: # pgmpy has an error when the number of features is 16. Unknown reason
        fea_subset = np.array(features)[fea_subsets[k]]
    else:
        fea_subset = features
        
    for root_node in fea_subset:
        df_train = pd.concat([X_train[fea_subset], y_train.rename(response_var)], axis=1)
        # df_train = pd.DataFrame(np.concatenate((y_train, X_train), axis=1), columns=[response_var] + features)
        
        # Learn the TAN structure from the data
        # Choose a random feature as the root node
        estimator = TreeSearch(df_train, root_node=root_node)
        dag = estimator.estimate(estimator_type='tan', class_node=response_var)
        
        # Construct Bayesian Network from the learned structure
        model = BayesianNetwork(dag.edges())
        
        # Parameterize the model using the training data
        model.fit(df_train, estimator=BayesianEstimator, prior_type='K2')
        
        # Make predictions on the test set using the trained TAN model
        X_test_df = pd.DataFrame(X_test, columns=features)
        y_pred = model.predict(X_test_df[fea_subset]).values
        
        
        accuracy = accuracy_score(y_test, y_pred)
        matrix = confusion_matrix(y_test, y_pred)
        
        accuracy_per_class = matrix.diagonal() / matrix.sum(axis=1)
        f1_macro = f1_score(y_test, y_pred, average='macro')
        precision = precision_score(y_test, y_pred, average='macro')
        recall = recall_score(y_test, y_pred, average='macro')
        
        print(f"TAN,{k},{accuracy:.03f},{",".join(f"{x:.3f}" for x in accuracy_per_class)},{f1_macro:.03f},{precision:.03f},{recall:.03f},{root_node}")
        

Building tree:   0%|          | 0/231.0 [00:00<?, ?it/s]

  0%|          | 0/24703 [00:00<?, ?it/s]

TAN,16,0.832,0.923,0.000,0.393,0.440,0.443,0.439,BMI_q_normal


Building tree:   0%|          | 0/231.0 [00:00<?, ?it/s]

  0%|          | 0/24703 [00:00<?, ?it/s]

...

  0%|          | 0/24703 [00:00<?, ?it/s]

TAN,16,0.832,0.923,0.000,0.394,0.441,0.444,0.439,DiffWalk


Building tree:   0%|          | 0/231.0 [00:00<?, ?it/s]

  0%|          | 0/24703 [00:00<?, ?it/s]

TAN,16,0.832,0.923,0.000,0.393,0.440,0.443,0.439,Sex


In [38]:
output = """
model,num fea,accuracy,accuracy 0,accuracy 1,accuracy 2,f1 macro,precision,recall,root node
TAN,8,0.837,0.938,0.000,0.336,0.432,0.445,0.425,BMI_q_normal
TAN,8,0.837,0.938,0.000,0.336,0.432,0.445,0.425,GenHlth_q_uniform
TAN,8,0.837,0.938,0.000,0.336,0.432,0.445,0.425,Age_q_uniform
TAN,8,0.837,0.938,0.000,0.336,0.432,0.445,0.425,Income_q_uniform
TAN,8,0.837,0.938,0.000,0.336,0.432,0.445,0.425,HighBP
TAN,8,0.837,0.938,0.000,0.336,0.432,0.445,0.425,HighChol
TAN,8,0.837,0.938,0.000,0.336,0.432,0.445,0.425,HeartDiseaseorAttack
TAN,8,0.837,0.938,0.000,0.336,0.432,0.445,0.425,HvyAlcoholConsump
TAN,10,0.836,0.936,0.000,0.346,0.433,0.445,0.427,BMI_q_normal
TAN,10,0.837,0.936,0.000,0.347,0.434,0.446,0.428,GenHlth_q_uniform
TAN,10,0.837,0.936,0.000,0.347,0.434,0.446,0.428,Age_q_uniform
TAN,10,0.837,0.936,0.000,0.347,0.434,0.446,0.428,Income_q_uniform
TAN,10,0.837,0.936,0.000,0.347,0.434,0.446,0.428,HighBP
TAN,10,0.837,0.936,0.000,0.347,0.434,0.446,0.428,HighChol
TAN,10,0.837,0.936,0.000,0.347,0.434,0.446,0.428,CholCheck
TAN,10,0.837,0.936,0.000,0.347,0.434,0.446,0.428,HeartDiseaseorAttack
TAN,10,0.836,0.936,0.000,0.346,0.433,0.445,0.427,HvyAlcoholConsump
TAN,10,0.837,0.936,0.000,0.346,0.434,0.446,0.427,Sex
TAN,12,0.835,0.931,0.000,0.367,0.437,0.445,0.433,BMI_q_normal
TAN,12,0.835,0.931,0.002,0.367,0.439,0.556,0.433,GenHlth_q_uniform
TAN,12,0.835,0.931,0.002,0.366,0.439,0.556,0.433,Age_q_uniform
TAN,12,0.835,0.931,0.002,0.366,0.439,0.556,0.433,Income_q_uniform
TAN,12,0.835,0.931,0.002,0.366,0.439,0.556,0.433,HighBP
TAN,12,0.835,0.931,0.002,0.366,0.439,0.556,0.433,HighChol
TAN,12,0.835,0.931,0.002,0.366,0.439,0.556,0.433,CholCheck
TAN,12,0.835,0.931,0.002,0.367,0.439,0.556,0.433,Stroke
TAN,12,0.835,0.931,0.002,0.366,0.439,0.556,0.433,HeartDiseaseorAttack
TAN,12,0.835,0.931,0.000,0.367,0.437,0.445,0.433,HvyAlcoholConsump
TAN,12,0.835,0.931,0.002,0.366,0.439,0.556,0.433,AnyHealthcare
TAN,12,0.835,0.931,0.002,0.366,0.439,0.556,0.433,DiffWalk
TAN,14,0.834,0.927,0.000,0.378,0.438,0.444,0.435,BMI_q_normal
TAN,14,0.834,0.928,0.002,0.378,0.440,0.611,0.436,GenHlth_q_uniform
TAN,14,0.834,0.928,0.002,0.378,0.440,0.611,0.436,Age_q_uniform
TAN,14,0.834,0.928,0.002,0.377,0.440,0.611,0.436,Education_coxbox
TAN,14,0.834,0.928,0.002,0.378,0.440,0.611,0.436,Income_q_uniform
TAN,14,0.834,0.928,0.002,0.378,0.440,0.611,0.436,HighBP
TAN,14,0.834,0.928,0.002,0.378,0.440,0.611,0.436,HighChol
TAN,14,0.834,0.928,0.002,0.378,0.440,0.611,0.436,CholCheck
TAN,14,0.834,0.928,0.002,0.378,0.440,0.611,0.436,Stroke
TAN,14,0.834,0.928,0.002,0.378,0.440,0.611,0.436,HeartDiseaseorAttack
TAN,14,0.834,0.928,0.002,0.378,0.440,0.611,0.436,PhysActivity
TAN,14,0.834,0.927,0.000,0.377,0.438,0.444,0.435,HvyAlcoholConsump
TAN,14,0.834,0.928,0.002,0.378,0.440,0.611,0.436,AnyHealthcare
TAN,14,0.834,0.928,0.002,0.378,0.440,0.611,0.436,DiffWalk
TAN,16,0.832,0.923,0.000,0.393,0.440,0.443,0.439,BMI_q_normal
TAN,16,0.832,0.923,0.000,0.394,0.441,0.443,0.439,MentHlth
TAN,16,0.832,0.923,0.000,0.394,0.441,0.443,0.439,PhysHlth_q_uniform
TAN,16,0.832,0.923,0.000,0.394,0.441,0.444,0.439,GenHlth_q_uniform
TAN,16,0.832,0.923,0.000,0.394,0.441,0.443,0.439,Age_q_uniform
TAN,16,0.832,0.923,0.000,0.394,0.441,0.444,0.439,Education_coxbox
TAN,16,0.832,0.923,0.000,0.394,0.441,0.443,0.439,Income_q_uniform
TAN,16,0.832,0.923,0.000,0.394,0.441,0.443,0.439,HighBP
TAN,16,0.832,0.923,0.000,0.394,0.441,0.443,0.439,HighChol
TAN,16,0.832,0.923,0.000,0.394,0.441,0.443,0.439,CholCheck
TAN,16,0.832,0.923,0.000,0.394,0.441,0.444,0.439,Smoker
TAN,16,0.832,0.923,0.000,0.394,0.441,0.444,0.439,Stroke
TAN,16,0.832,0.923,0.000,0.394,0.441,0.443,0.439,HeartDiseaseorAttack
TAN,16,0.832,0.923,0.000,0.394,0.441,0.443,0.439,PhysActivity
TAN,16,0.832,0.923,0.000,0.393,0.441,0.443,0.439,Fruits
TAN,16,0.832,0.923,0.000,0.393,0.441,0.443,0.439,Veggies
TAN,16,0.832,0.923,0.000,0.394,0.441,0.444,0.439,HvyAlcoholConsump
TAN,16,0.832,0.923,0.000,0.394,0.441,0.443,0.439,AnyHealthcare
TAN,16,0.832,0.923,0.000,0.394,0.441,0.443,0.439,NoDocbcCost
TAN,16,0.832,0.923,0.000,0.394,0.441,0.444,0.439,DiffWalk
TAN,16,0.832,0.923,0.000,0.393,0.440,0.443,0.439,Sex
"""

stats = pd.read_csv(StringIO(output), delimiter=",")
best_tan_models = stats.loc[stats.groupby('num fea')['f1 macro'].idxmax()]
best_tan_models

,model,num fea,accuracy,accuracy 0,accuracy 1,accuracy 2,f1 macro,precision,recall,root node
0,TAN,8,0.837,0.938,0.000,0.336,0.432,0.445,0.425,BMI_q_normal
9,TAN,10,0.837,0.936,0.000,0.347,0.434,0.446,0.428,GenHlth_q_uniform
19,TAN,12,0.835,0.931,0.002,0.367,0.439,0.556,0.433,GenHlth_q_uniform
31,TAN,14,0.834,0.928,0.002,0.378,0.440,0.611,0.436,GenHlth_q_uniform
45,TAN,16,0.832,0.923,0.000,0.394,0.441,0.443,0.439,MentHlth


In [133]:

# print(best_models)
print("model,accuracy,accuracy 0,accuracy 1,accuracy 2,f1_macro,precision,recall")
for index, row in best_models.iterrows():
    model_name = row['model']
    if model_name != "GradientBoostingClassifier":
        continue 
    print(row)
    
    subset_features = np.array(features)[fea_subsets[int(row['num fea'])]]
    kwargs = row['best parameters'].replace("OrderedDict(", '').replace("})", '}')
    kwargs = ast.literal_eval(kwargs)
    if model_name == 'Logistic regression': 
        model = LogisticRegression(solver='newton-cholesky', **kwargs)
    elif model_name == 'Discriminant analysis (not svd)': 
        model = LinearDiscriminantAnalysis(**kwargs)
    elif model_name == 'Discriminant analysis (svd)': 
        model = LinearDiscriminantAnalysis(store_covariance=True, **kwargs)
    elif model_name == 'ComplementNB': 
        model = ComplementNB(**kwargs)
    elif model_name == 'QuadraticDiscriminantAnalysis': 
        model = QuadraticDiscriminantAnalysis(**kwargs)
    elif model_name == "GradientBoostingClassifier":
        model = GradientBoostingClassifier()
    else:
        raise ValueError("Unknown model")
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)  # You can adjust n_splits as needed
    accuracies = []
    f1_macros = []
    precisions = []
    recalls = []
    accuracies_per_class = []
    
    for i, (train_index, test_index) in enumerate(kf.split(X_train)):
        X_train_fold, y_train_fold = X_train.iloc[train_index], y_train.iloc[train_index]
        X_test_fold, y_test_fold = X_train.iloc[test_index], y_train.iloc[test_index]
        
        if model_name == 'ComplementNB':
            mmt = MinMaxScaler()
            X_train_fold = mmt.fit_transform(X_train_fold)
            X_test_fold = mmt.transform(X_test_fold)  # Apply the same scaler for test fold
            model.fit(X_train_fold[subset_features], y_train_fold)  # Fit the model on the transformed data
        else:
            model.fit(X_train_fold[subset_features], y_train_fold) 
        
        y_pred = model.predict(X_test_fold[subset_features])
        
        # Calculate the metrics
        accuracy = accuracy_score(y_test_fold, y_pred)
        accuracies.append(accuracy)
        
        matrix = confusion_matrix(y_test_fold, y_pred)
        accuracy_per_class = matrix.diagonal() / matrix.sum(axis=1)
        accuracies_per_class.append(accuracy_per_class)
        
        f1_macro = f1_score(y_test_fold, y_pred, average='macro')
        f1_macros.append(f1_macro)
        
        precision = precision_score(y_test_fold, y_pred, average='macro')
        precisions.append(precision)
        
        recall = recall_score(y_test_fold, y_pred, average='macro')
        recalls.append(recall)
    
    # Print the metrics
    print(f"{model_name},{np.mean(accuracies):.03f},{",".join(f"{x:.3f}" for x in np.mean(accuracies_per_class, axis=0))},{np.mean(f1_macros):.03f},{np.mean(precisions):.03f},{np.mean(recalls):.03f}")
    

model,accuracy,accuracy 0,accuracy 1,accuracy 2,f1_macro,precision,recall
it                                                                 3
model                                     GradientBoostingClassifier
num fea                                                           14
best parameters    OrderedDict({'learning_rate': 0.25284262311045...
score                                                         0.4033
elapsed time                                                  1474.3
Name: 28, dtype: object
GradientBoostingClassifier,0.850,0.977,0.000,0.192,0.401,0.473,0.390


In [42]:
output = """model,accuracy,accuracy 0,accuracy 1,accuracy 2,f1_macro,precision,recall
ComplementNB,0.790,0.885,0.091,0.305,0.425,0.433,0.427
Discriminant analysis (not svd),0.818,0.900,0.000,0.430,0.437,0.431,0.443
Discriminant analysis (svd),0.845,0.968,0.000,0.212,0.405,0.458,0.393
Logistic regression,0.687,0.700,0.167,0.678,0.436,0.438,0.515
QuadraticDiscriminantAnalysis,0.772,0.830,0.010,0.521,0.428,0.424,0.454
TAN,0.832,0.923,0.000,0.394,0.441,0.443,0.439
"""
stats = pd.read_csv(StringIO(output), delimiter=",")
stats

,model,accuracy,accuracy 0,accuracy 1,accuracy 2,f1_macro,precision,recall
0,ComplementNB,0.790,0.885,0.091,0.305,0.425,0.433,0.427
1,Discriminant analysis (not svd),0.818,0.900,0.000,0.430,0.437,0.431,0.443
2,Discriminant analysis (svd),0.845,0.968,0.000,0.212,0.405,0.458,0.393
3,Logistic regression,0.687,0.700,0.167,0.678,0.436,0.438,0.515
4,QuadraticDiscriminantAnalysis,0.772,0.830,0.010,0.521,0.428,0.424,0.454
5,TAN,0.832,0.923,0.000,0.394,0.441,0.443,0.439


In [134]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
import ast



# Add the model_name and kwargs into df_best_models
new_row = pd.DataFrame({
    'model': ['TAN'],  # wrap in a list to indicate a row
    'best parameters': ["OrderedDict({})"] , # use OrderedDict object directly
    'num fea': X.shape[1]
})

print(best_models.shape)
# Concatenate the new row to the existing df_best_models DataFrame
complete_best_models = pd.concat([new_row, best_models], ignore_index=True)

print(complete_best_models.shape)

print("model,accuracy,accuracy 0,accuracy 1,accuracy 2,f1_macro,precision,recall")
for index, row in complete_best_models.iloc[::-1].iterrows():
    model_name = row['model']
    if model_name != "GradientBoostingClassifier":
        continue 
    kwargs = row['best parameters'].replace("OrderedDict(", '').replace("})", '}')
    kwargs = ast.literal_eval(kwargs)
    subset_features = np.array(features)[fea_subsets[int(row['num fea'])]] if int(row['num fea']) in fea_subsets else features
    
    if model_name == 'Logistic regression': 
        model = LogisticRegression(solver='newton-cholesky', **kwargs)
    elif model_name == 'Discriminant analysis (not svd)': 
        model = LinearDiscriminantAnalysis(**kwargs)
    elif model_name == 'Discriminant analysis (svd)': 
        model = LinearDiscriminantAnalysis(store_covariance=True, **kwargs)
    elif model_name == 'ComplementNB': 
        model = ComplementNB(**kwargs)
    elif model_name == 'QuadraticDiscriminantAnalysis': 
        model = QuadraticDiscriminantAnalysis(**kwargs)
    elif model_name == 'TAN': 
        df_train = pd.concat([X_train[subset_features], y_train.rename(response_var)], axis=1)
        estimator = TreeSearch(df_train, root_node='MentHlth')
        dag = estimator.estimate(estimator_type='tan', class_node=response_var)
    elif model_name == "GradientBoostingClassifier":
        model = GradientBoostingClassifier()
    else:
        raise ValueError(f"Unknown model {model_name}")
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)  # You can adjust n_splits as needed
    accuracies = []
    f1_macros = []
    precisions = []
    recalls = []
    accuracies_per_class = []
    
    for i, (train_index, test_index) in enumerate(kf.split(X_train)):
        X_train_fold, y_train_fold = X_train.iloc[train_index], y_train.iloc[train_index]
        X_test_fold, y_test_fold = X_train.iloc[test_index], y_train.iloc[test_index]
        
        if model_name == 'ComplementNB':
            mmt = MinMaxScaler()
            X_train_fold = mmt.fit_transform(X_train_fold[subset_features])
            X_test_fold = mmt.transform(X_test_fold[subset_features])  # Apply the same scaler for test fold
            model.fit(X_train_fold, y_train_fold)  # Fit the model on the transformed data
            y_pred = model.predict(X_test_fold)
        elif model_name == "TAN":
            # Parameterize the model using the training data
            model = BayesianNetwork(dag.edges())
            model.fit(df_train, estimator=BayesianEstimator, prior_type='K2', n_jobs=-1)
            y_pred = model.predict(X_test_fold[subset_features])
        else:
            model.fit(X_train_fold[subset_features], y_train_fold) 
            y_pred = model.predict(X_test_fold[subset_features])
        
        # Calculate the metrics
        accuracy = accuracy_score(y_test_fold, y_pred)
        accuracies.append(accuracy)
        
        matrix = confusion_matrix(y_test_fold, y_pred)
        accuracy_per_class = matrix.diagonal() / matrix.sum(axis=1)
        accuracies_per_class.append(accuracy_per_class)
        
        f1_macro = f1_score(y_test_fold, y_pred, average='macro')
        f1_macros.append(f1_macro)
        
        precision = precision_score(y_test_fold, y_pred, average='macro')
        precisions.append(precision)
        
        recall = recall_score(y_test_fold, y_pred, average='macro')
        recalls.append(recall)
    
    # Print the metrics
    print(f"{model_name},{np.mean(accuracies):.03f},{",".join(f"{x:.3f}" for x in np.mean(accuracies_per_class, axis=0))},{np.mean(f1_macros):.03f},{np.mean(precisions):.03f},{np.mean(recalls):.03f}")


"""
model,accuracy,accuracy 0,accuracy 1,accuracy 2,f1_macro,precision,recall
TAN,0.833,0.923,0.003,0.395,0.443,0.537,0.440
QuadraticDiscriminantAnalysis,0.793,0.846,0.000,0.574,0.442,0.426,0.473
Logistic regression,0.689,0.701,0.154,0.685,0.436,0.437,0.513
Discriminant analysis (svd),0.845,0.967,0.000,0.220,0.407,0.457,0.396
Discriminant analysis (not svd),0.817,0.892,0.000,0.466,0.442,0.433,0.453
ComplementNB,0.770,0.859,0.129,0.312,0.422,0.432,0.433
"""

(6, 6)
(7, 6)
model,accuracy,accuracy 0,accuracy 1,accuracy 2,f1_macro,precision,recall
GradientBoostingClassifier,0.850,0.977,0.000,0.192,0.401,0.473,0.390


'\nmodel,accuracy,accuracy 0,accuracy 1,accuracy 2,f1_macro,precision,recall\nTAN,0.833,0.923,0.003,0.395,0.443,0.537,0.440\nQuadraticDiscriminantAnalysis,0.793,0.846,0.000,0.574,0.442,0.426,0.473\nLogistic regression,0.689,0.701,0.154,0.685,0.436,0.437,0.513\nDiscriminant analysis (svd),0.845,0.967,0.000,0.220,0.407,0.457,0.396\nDiscriminant analysis (not svd),0.817,0.892,0.000,0.466,0.442,0.433,0.453\nComplementNB,0.770,0.859,0.129,0.312,0.422,0.432,0.433\n'

In [135]:




# Add the model_name and kwargs into df_best_models
new_row = pd.DataFrame({
    'model': ['TAN'],  # wrap in a list to indicate a row
    'best parameters': ["OrderedDict({})"] , # use OrderedDict object directly
    'num fea': X.shape[1]
})

print(best_models.shape)
# Concatenate the new row to the existing df_best_models DataFrame
complete_best_models = pd.concat([new_row, best_models], ignore_index=True)

print(complete_best_models.shape)

print("model,accuracy,accuracy 0,accuracy 1,accuracy 2,f1_macro,precision,recall")

for index, row in complete_best_models.iterrows():
    model_name = row['model']
    if model_name != "GradientBoostingClassifier":
        continue 
    kwargs = row['best parameters'].replace("OrderedDict(", '').replace("})", '}')
    kwargs = ast.literal_eval(kwargs)
    subset_features = np.array(features)[fea_subsets[int(row['num fea'])]] if int(row['num fea']) in fea_subsets else features
    
    if model_name == 'Logistic regression': 
        model = LogisticRegression(solver='newton-cholesky', **kwargs)
    elif model_name == 'Discriminant analysis (not svd)': 
        model = LinearDiscriminantAnalysis(**kwargs)
    elif model_name == 'Discriminant analysis (svd)': 
        model = LinearDiscriminantAnalysis(store_covariance=True, **kwargs)
    elif model_name == 'ComplementNB': 
        model = ComplementNB(**kwargs)
    elif model_name == 'QuadraticDiscriminantAnalysis': 
        model = QuadraticDiscriminantAnalysis(**kwargs)
    elif model_name == 'TAN': 
        pass
    elif model_name == "GradientBoostingClassifier":
        model = GradientBoostingClassifier()
    else:
        raise ValueError("Unknown model")
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)  # You can adjust n_splits as needed
    accuracies = []
    f1_macros = []
    precisions = []
    recalls = []
    accuracies_per_class = []
    
    for i, (train_index, test_index) in enumerate(kf.split(X_train)):
        X_train_fold, y_train_fold = X_train.iloc[train_index], y_train.iloc[train_index]
        X_test_fold, y_test_fold = X_train.iloc[test_index], y_train.iloc[test_index]
        
        # Apply Random Undersampling to balance the classes in the training fold
        undersampler = RandomUnderSampler(random_state=rng)
        X_train_fold_resampled, y_train_fold_resampled = undersampler.fit_resample(X_train_fold[subset_features], y_train_fold)
        # X_train_fold_resampled, y_train_fold_resampled = X_train_fold[subset_features], y_train_fold
    
        if model_name == 'ComplementNB':
            mmt = MinMaxScaler()
            X_train_fold_resampled = mmt.fit_transform(X_train_fold_resampled)
            X_test_fold = mmt.fit_transform(X_test_fold[subset_features])  # Apply the same scaler for test fold
            model.fit(X_train_fold_resampled, y_train_fold_resampled)  # Fit the model on the transformed data
            y_pred = model.predict(X_test_fold)  # Make predictions on the original test fold
        elif model_name == "TAN":
            # Parameterize the model using the training data
            print(X_train_fold_resampled.__class__)
            df_train = pd.concat([X_train_fold_resampled, y_train_fold_resampled.rename(response_var)], axis=1)
            df_train.reset_index(drop=True, inplace=True)
            
            estimator = TreeSearch(df_train, root_node='MentHlth')
            dag = estimator.estimate(estimator_type='tan', class_node=response_var)
            model = BayesianNetwork(dag.edges())
            model.fit(df_train, estimator=BayesianEstimator, prior_type='K2', n_jobs=-1)
            y_pred = model.predict(data=X_test_fold[subset_features],n_jobs=-1)  # Make predictions on the original test fold
        else:
            model.fit(X_train_fold_resampled, y_train_fold_resampled) 
            y_pred = model.predict(X_test_fold[subset_features])  # Make predictions on the original test fold
        
        
        
        
        # Calculate the metrics
        accuracy = accuracy_score(y_test_fold, y_pred)
        accuracies.append(accuracy)
        
        matrix = confusion_matrix(y_test_fold, y_pred)
        accuracy_per_class = matrix.diagonal() / matrix.sum(axis=1)
        accuracies_per_class.append(accuracy_per_class)
        
        f1_macro = f1_score(y_test_fold, y_pred, average='macro')
        f1_macros.append(f1_macro)
        
        precision = precision_score(y_test_fold, y_pred, average='macro')
        precisions.append(precision)
        
        recall = recall_score(y_test_fold, y_pred, average='macro')
        recalls.append(recall)
    
    # Print the metrics
    print(f"\r{model_name},{np.mean(accuracies):.03f},{",".join(f"{x:.3f}" for x in np.mean(accuracies_per_class, axis=0))},{np.mean(f1_macros):.03f},{np.mean(precisions):.03f},{np.mean(recalls):.03f}")




(6, 6)
(7, 6)
model,accuracy,accuracy 0,accuracy 1,accuracy 2,f1_macro,precision,recall
GradientBoostingClassifier,0.612,0.620,0.343,0.598,0.416,0.445,0.521


In [112]:
output = """model,accuracy,accuracy 0,accuracy 1,accuracy 2,f1_macro,precision,recall
QuadraticDiscriminantAnalysis,0.556,0.521,0.147,0.824,0.375,0.420,0.497
Logistic regression,0.677,0.695,0.209,0.633,0.433,0.440,0.512
Discriminant analysis (svd),0.650,0.665,0.244,0.610,0.427,0.445,0.506
Discriminant analysis (not svd),0.642,0.661,0.261,0.580,0.422,0.442,0.500
ComplementNB,0.728,0.815,0.252,0.264,0.407,0.434,0.444
"""

stats = pd.read_csv(StringIO(output), delimiter=",")
print(stats.columns)
best_models = stats.loc[stats['f1_macro'].idxmax()]
best_models

Index(['model', 'accuracy', 'accuracy 0', 'accuracy 1', 'accuracy 2',
       'f1_macro', 'precision', 'recall'],
      dtype='object')


model         Logistic regression
accuracy                    0.677
accuracy 0                  0.695
accuracy 1                  0.209
accuracy 2                  0.633
f1_macro                    0.433
precision                    0.44
recall                      0.512
Name: 1, dtype: object